# Predict Helix Capping Residues #

The goal is to identify residues just before an alpha helix begins or the residues just after the helix ends. This will improve secondary structure predictors becuase they often extend too far or do not start at the right place. 

The CapsDB has annoted sequences of structures of helix capping residues that can be used to train a deep nueral net. We will use a Bidirectional LSTM using phi/psi features to see if it will those will be good predictors.

## 1. Download data ##

## 2. Generate Features ##
### MMTF Pyspark Imports ###

In [6]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from mmtfPyspark.io import mmtfReader
from mmtfPyspark.webfilters import Pisces
from mmtfPyspark.filters import ContainsLProteinChain
from mmtfPyspark.mappers import StructureToPolymerChains
from mmtfPyspark.ml import ProteinSequenceEncoder
import numpy as np
import pandas as pd

### Custom imports ###

In [ ]:
import secondaryStructureExtractorFull
#import mmtfToASA
import os
os.getcwd()

### Configure Spark Context ###

In [ ]:
spark = SparkSession.builder.master("local[8]").appName("DeepCap").getOrCreate()

### Filter out chains not in CapsDB ###

In [ ]:
from pyspark.sql import SQLContext
from pyspark.sql.functions import concat, col, lit, array_contains

sqlContext = SQLContext(spark)
capsdb = sqlContext.read.parquet('caps_descriptors.parquet')
capsdb_pdbs = capsdb.select(concat(upper(col("pdbId")), lit("."), col("chain")).alias("id")).drop_duplicates()


In [ ]:
capsdb_pdbs.count()

In [ ]:
#from pyspark.sql.types import BooleanType

# def hasHelixCapInfo(enc_obj, ids):
#     return(enc_obj[0] in ids)

# def udf_hasHelixCapInfo(ids):
#     return udf(lambda t: hasHelixCapInfo(t, ids))
#     # I'm not sure how to call these successfully - Sean

capsdb_ids = set()
[capsdb_ids.add(list[0]) for list in capsdb_pdbs.select("id").collect()]

class HasHelixCapInfo(object):
    '''This filter returns true if the structure is in CAPSDB'''
    def __call__(self, t):
        return(t[0] in capsdb_ids)
    # This works but I don't like that it refers to a global variable: not sure how to change that right now - Sean


### Read MMTF File and get a set of L-protein chains ###

In [ ]:
if not os.path.isdir("full"):
    !wget https://mmtf.rcsb.org/v1.0/hadoopfiles/full.tar && tar -xvf full.tar
        
if not os.path.isdir("reduced"):
    !wget https://mmtf.rcsb.org/v1.0/hadoopfiles/reduced.tar && tar -xvf reduced.tar

In [ ]:
pdb = mmtfReader.read_sequence_file('full') \
    .flatMap(StructureToPolymerChains()) \
    .filter(ContainsLProteinChain()) \
    .filter(HasHelixCapInfo())
#pdb.count()
#pdb.take(2)
#?pdb

### Get Torsion angle and secondary structure info ###

In [ ]:
#from mmtfPyspark.datasets import secondaryStructureExtractor

data = secondaryStructureExtractorFull.get_dataset(pdb).toPandas()
#data = secondaryStructureExtractor.get_dataset(pdb).toPandas()

In [ ]:
# Read output of above get_dataset operation from parquet file
parquetPath = '/home/ec2-user/SageMaker/ProteinFragmenter/datacaps.parquet'
dataframe = sqlContext.read.parquet(parquetPath)
data = dataframe.toPandas()
data = data.drop('__index_level_0__', axis=1)

In [ ]:
data.head(10)

In [ ]:
df1 = capsdb.toPandas()
df = pd.merge(data, df1, left_on=('pdbId','chain'), right_on=('pdbid','chain'), how='inner')
df = df[['pdbId', 'chain', 'resi', 'resn', 'phi', 'psi', 'startcap', 'endcap']]


### Create labels

In [ ]:
df['is_cap'] = df.apply(lambda x: 1 if (x['resi'] >= x['startcap'] and x['resi'] <= x['endcap']) else 0, axis=1)
df_caps = df.groupby(["pdbId", "chain", "resi"])['is_cap'].max().reset_index()

In [ ]:
data_caps = pd.merge(data, df_caps, left_on=('pdbId','chain', 'resi'), right_on=('pdbId','chain', 'resi'), how='inner')

In [ ]:
def is_cap(pdbId, chain, resi, is_cap):
    if is_cap == 1:
        return([1,0])
    elif is_cap == 0:
        return([0,1])
    else:
        raise ValueError("is_cap must be 0 or 1")

In [ ]:
groups = data_caps.groupby(["pdbId", "chain"])
                           # num pdbs,    max len of seqs, num features
training_data = np.zeros((groups.ngroups, 5000, 2), dtype=float)
                           # num pdbs,    max len of seqs, length of one-hot encoded target
truth = np.zeros((groups.ngroups, 5000, 2), dtype=int)
truth_lagged = np.zeros((groups.ngroups, 5000, 2), dtype=int)


In [ ]:
data_caps.head()

In [ ]:
for i, ((pdbid, chain), group) in enumerate(groups):
    for j, featuretuple in enumerate(group.itertuples()):
        if j>=5000: break
        training_data[i,j,:] = (featuretuple.phi, featuretuple.psi)
        truth[i,j,:] = is_cap(featuretuple.pdbId, featuretuple.chain, featuretuple.resi, featuretuple.is_cap)
        if (j > 0):
            truth_lagged[i,j-1,:] = truth[i,j,:]
        

In [ ]:
#training_data[10]
#truth[2]
truth[8,:,0].sum()
#truth.shape
#training_data.shape
#os.getcwd()

### Write features, labels to H5 file ###

In [ ]:
import h5py
os.remove('features.h5')
os.remove('truth.h5')
os.remove('truthlag.h5')

h5f = h5py.File('features.h5', 'w')
h5f.create_dataset('training_data', data=training_data)
h5f.close()

h5tr = h5py.File('truth.h5', 'w')
h5tr.create_dataset('truth', data=truth)
h5tr.close()

h5trl = h5py.File('truthlag.h5', 'w')
h5trl.create_dataset('truthlag', data=truth_lagged)
h5trl.close()

#os.remove("features.h5")

### Terminate Spark ###

In [ ]:
spark.stop()

## 4. Build Bidirectional LSTM ##

In [33]:
def create_and_train_model(num_features, num_outputs=2, latent_dim=5):
    """Create a Seq2Seq Bidirectional LSTM
    From: https://blog.keras.io/a-ten-minute-introduction-to-sequence-to-sequence-learning-in-keras.html
    
    Parameters
    ----------
    num_features : int
        The number of features in your trianing data
    num_outputs : int
        Number of outputs to rpedict, i.e. number of classes or 2 for binary
        
    Returns
    -------
    A new Keras Seq2Seq Bidirectional LSTM
    """
    
    # Define an input sequence and process it.
    encoder_inputs = Input(shape=(None, num_features))
    encoder = LSTM(latent_dim, return_state=True)
    #encoder = Bidirectional(LSTM(latent_dim, return_state=True))
    encoder_outputs, state_h, state_c = encoder(encoder_inputs)
    #encoder_outputs, forward_h, forward_c, backward_h, backward_c = encoder(encoder_inputs)
    #state_h = Concatenate()([forward_h, backward_h])
    #state_c = Concatenate()([forward_c, backward_c])
    
    # We discard `encoder_outputs` and only keep the states.
    encoder_states = [state_h, state_c]

    # Set up the decoder, using `encoder_states` as initial state.
    #decoder_inputs = Input(shape=(None, num_outputs))
        
    # We set up our decoder to return full output sequences,
    # and to return internal states as well. We don't use the
    # return states in the training model, but we will use them in inference.
    #decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
    #decoder_lstm = Bidirectional(LSTM(latent_dim, return_sequences=True, return_state=True))
    #decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
    
    decoder_inputs = Input(shape=(None, num_outputs))
    decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
    #decoder_lstm = LSTM(latent_dim * 2, return_sequences=True, return_state=True)
    decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
    
    decoder_dense = Dense(num_outputs, activation='softmax')
    decoder_outputs = decoder_dense(decoder_outputs)
    
    model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    
    # Train model
    encoder_input_data = HDF5Matrix('features.h5', 'training_data')
    decoder_input_data = HDF5Matrix('truth.h5', 'truth')
    decoder_target_data = HDF5Matrix('truthlag.h5', 'truthlag')
    
    #Automicatlly determine batch sizes, train/test splits
    model.fit([encoder_input_data, decoder_input_data], decoder_target_data, shuffle="batch", epochs=10) #, callbacks=get_callbacks(model))
    #print(model.get_weights())
    #with open('model_weights.txt', 'w') as f:
    #    f.write(print(model.get_weights()))
    for (i, obj) in enumerate(model.get_weights()):
        obj.tofile("weights_layer{0}".format(str(i)))
    #.tofile("test2.txt")
    
    # Create encoder/decoder model
    encoder_model = Model(encoder_inputs, encoder_states)

    decoder_state_input_h = Input(shape=(latent_dim,))
    decoder_state_input_c = Input(shape=(latent_dim,))
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

    decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
    decoder_states = [state_h, state_c]

    decoder_outputs = decoder_dense(decoder_outputs)
    decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)
    
    return encoder_model, decoder_model

In [ ]:
def get_callbacks(model_obj, logging_file="training.log", verbosity=1, early_stopping_patience=None):
    callbacks = list()
    callbacks.append(ModelCheckpoint(model_obj, save_best_only=True))
    callbacks.append(CSVLogger(logging_file, append=True))
#    callbacks.append(LossHistory('losshistory.log', append=True))
    callbacks.append(TensorBoard(log_dir='./logs', histogram_freq=0, batch_size=32, write_graph=True, write_grads=True, write_images=False, embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None, embeddings_data=None, update_freq='batch'))
#     if learning_rate_epochs:
#         callbacks.append(LearningRateScheduler(partial(step_decay, initial_lrate=initial_learning_rate,
#                                                        drop=learning_rate_drop, epochs_drop=learning_rate_epochs)))
#     else:
#         callbacks.append(ReduceLROnPlateau(factor=learning_rate_drop, patience=learning_rate_patience,
#                                            verbose=verbosity))
#     if early_stopping_patience:
#         callbacks.append(EarlyStopping(verbose=verbosity, patience=early_stopping_patience))
    return callbacks

In [ ]:
#encoder_input_data = HDF5Matrix('features.h5', 'training_data')
training_data.tofile("test.txt")

In [ ]:
# get subset of training data, if needed for faster debugging

#training_data = HDF5Matrix('features.h5', 'training_data')
import h5py
hf = h5py.File('full_dataset_features/features.h5', 'r')
training_data = hf.get('training_data').value
training_data = training_data[0:128,:,:]
hf.close()

hf = h5py.File('full_dataset_features/truth.h5', 'r')
truth = hf.get('truth').value
truth = truth[0:128,:,:]
hf.close()

hf = h5py.File('full_dataset_features/truthlag.h5', 'r')
truth_lagged = hf.get('truthlag').value
truth_lagged = truth_lagged[0:128,:,:]
hf.close()

In [ ]:
#import numpy
#numpy.set_printoptions(threshold=numpy.nan)


In [ ]:
import tensorflow as tf
import keras
from keras.utils.io_utils import HDF5Matrix
from keras.layers import Input, Dense, Bidirectional, LSTM, Concatenate
from keras.models import Model
from keras.callbacks import ModelCheckpoint, CSVLogger, TensorBoard

# class LossHistory(keras.callbacks.Callback):
#     def on_train_begin(self, logs={}):
#         self.losses = []

#     def on_batch_end(self, batch, logs={}):
#         self.losses.append(logs.get('loss'))

#encoder_inputs, encoder_states, decoder_inputs, decoder_lstm, decoder_dense, model = train()
#model = train()
encoder_model, decoder_model = create_and_train_model(2)

Epoch 1/10


In [23]:
encoder_model.save_weights("encoder_weights")
decoder_model.save_weights("decoder_weights")

In [ ]:
# load json and create model
json_file = open('model_StS_BiLSTM.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = keras.models.model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model_StS_BiLSTM.h5")
print("Loaded model from disk")

In [59]:
np.fromfile("weights_layer7")

array([2.24711749e+307])

In [37]:
weights0

array([ 1.22199549e-006, -1.07067662e-005, -3.37417133e-007,
        1.67356665e-005,  6.53985916e-007,  2.24711749e+307,
        2.24711749e+307, -5.06000970e-012,  1.29100576e-017,
        2.70294593e-006, -4.85684593e-008,  1.20094240e-010,
       -1.87995487e-006, -1.16249375e-005, -3.17621879e-006,
        2.24711749e+307,  2.24711749e+307, -3.75207243e-009,
        1.70528141e-006, -9.35567296e-007])

In [47]:
def decode_sequence(input_seq, encoder_model, decoder_model):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    print(states_value)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, 2))
    # Populate the first character of target sequence with the start residue.
    target_seq[0, 0, :] = [0, 1] # decoder tokens one-hot encoding hardcoded here

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_chain = []
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        decoded_chain.append(sampled_token_index)

        # Exit condition: either hit max length
        # or find stop character.
        if len(decoded_chain) > 5000:
            stop_condition = True

        # Update the target sequence (of length 1).
        sampled_residue = [0, 0]
        sampled_residue[sampled_token_index] = 1
        target_seq = np.zeros((1, 1, 2))
        target_seq[0, 0, :] = sampled_residue

        # Update states
        states_value = [h, c]

    return decoded_chain

In [ ]:
# To Sean: ran til here

In [48]:
#training_data = HDF5Matrix('features.h5', 'training_data')
import h5py
hf = h5py.File('features.h5', 'r')
df_training = hf.get('training_data').value
hf.close()

# import h5py
# hf = h5py.File('truth.h5', 'r')
# df_truth = hf.get('truth').value
# hf.close()

/home/ec2-user/anaconda3/envs/myBinder-environment/lib/python3.6/site-packages/h5py/_hl/dataset.py:313: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  "Use dataset[()] instead.", H5pyDeprecationWarning)


In [ ]:
#df_training.shape
#df_training.shape
df_truth[2]

In [49]:
temp = df_training[4:5,:,:]

In [ ]:
temp.shape

In [50]:
result = decode_sequence(temp, encoder_model, decoder_model)

[array([[nan, nan, nan, nan, nan]], dtype=float32), array([[nan, nan, nan, nan, nan]], dtype=float32)]


In [51]:
i = 0
for j in result:
    i += j
i

0

In [ ]:
#encoder_model.get_layer()
#encoder_model.layers
#decoder_model.layers